In [5]:
from Functions import *

In [1]:
# SQL queries


# Create new table
sql1 = """
        CREATE TABLE moscow_poligons(id SERIAL PRIMARY KEY, house TEXT)
        """
# POINT, POLYGON, LINEWAY
sql2 = """
        SELECT AddGeometryColumn('moscow_buildings', 'polygon', 4326, 'POLYGON', 2)
        """
sql3 = """
        CREATE INDEX moscow_buildings_idx ON moscow_buildings USING gist(polygon);
        """
sql4 = """
        INSERT INTO buildings(street, house, point)
        VALUES ('%s', '%s', st_geomfromtext('%s', 4326))
        """
sql5 = """
        SELECT "addr:housenumber", building
        FROM planet_osm_polygon
        WHERE building is not Null and
        ST_Contains(way, st_transform(st_geomfromtext('%s', 4326), 3857))
        LIMIT 100
        """
sql6 = """
        SELECT street, house, ST_AsEWKT(ST_Transform(point, 4326)) 
        FROM buildings
        LIMIT 10
        """
sql7 = """
        INSERT INTO streets(name, highway, z_order, way)
        SELECT t1.name, t1.highway, t1.z_order, ST_Transform(t1.way, 4326)
        FROM planet_osm_line as t1, cities as t2
        WHERE t2.name = '%s' and t1.name is not Null and t1.highway is not Null and
                ST_Contains(t2.polygon, ST_Transform(t1.way, 4326))
        """
sql8 = """
        INSERT INTO cities(name, polygon)
        SELECT DISTINCT ON (name) name, ST_Transform(way, 4326)
        FROM planet_osm_polygon WHERE place = 'city';
        """
sql9 = """
        INSERT INTO moscow_poligons(house, polygon)
        SELECT "addr:housenumber", ST_Transform(t1.way, 4326)
        FROM planet_osm_polygon as t1, cities as t2
        WHERE building is not Null and "addr:housenumber" is not Null and 
                ST_Contains(t2.polygon, ST_Transform(t1.way, 4326))
        """
sql10 = """
        INSERT INTO streets(name, highway, z_order, way)
        SELECT t1.name, t1.highway, t1.z_order, ST_Transform(t1.way, 4326)
        FROM planet_osm_line as t1, cities as t2
        WHERE t2.name = '%s' and t1.name is not Null and t1.highway is not Null and
              ST_Contains(t2.polygon, ST_Transform(t1.way, 4326))
        """
sql11 = """
        SELECT count(*)
        FROM moscow_buildings
        """

# information shema
sql12 = """
        SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = 'planet_osm_polygon'
        """
sql13 = """ 
        SELECT osm_id, ST_AsEWKT(ST_Transform(way, 4326))
        FROM planet_osm_polygon as t1, cities as t2
        WHERE building = '%s' and "addr:housenumber" is not Null
        LIMIT 1
        """
sql14 = """
        SELECT AddGeometryColumn('moscow_poligons', 'centroid', 4326, 'POINT', 2) 
        """
sql15 = """
        CREATE INDEX moscow_poligons_idx_1 ON moscow_poligons USING gist(centroid);
        """
sql16 = """
        UPDATE moscow_poligons
        SET lon = %f, lat = 
        """
sql17 = """
        SELECT DropGeometryColumn ('moscow_poligons','centroid');
        """ 
sql18 = """
        ALTER TABLE moscow_poligons
        ADD lat FLOAT
        """
sql19 = """
        SELECT lon, lat
        FROM moscow_poligons
        WHERE id = 152760
        """
sql20 = """
        SELECT street, house, ST_AsEWKT(ST_Transform(point, 4326)) 
        FROM buildings
        WHERE id = 34956 
        """
sql21 = """
        SELECT count(*)
        FROM moscow_poligons
        WHERE house is Null
        """
sql22 = "DROP INDEX moscow_poligons_idx_1"

sql23 = "DROP TABLE Moscow_buildings"
sql24 = "CREATE TABLE Moscow_buildings(id SERIAL PRIMARY KEY, street TEXT, house TEXT)"
sql25 = """
        INSERT INTO moscow_buildings (street, house, polygon)
        VALUES ('%s', '%s', '%s')
        """
sql26 = "DELETE FROM moscow_buildings"

# Получить здания Москвы с тагами улиц
sql27 = """
        INSERT INTO moscow_buildings (street, house, polygon)
        SELECT tags, "addr:housenumber", ST_Transform(way, 4326) 
        FROM planet_osm_polygon as t1, cities as t2
        WHERE t2.name = '%s' and "addr:housenumber" is not Null and building is not Null and
               ST_Contains(t2.polygon, ST_Transform(t1.way, 4326))
        """

# Получить улицы из moscow_buildings в виде тагов
sql28 = """
        SELECT id, street
        FROM moscow_buildings
        """

# Обновить таги на обработанные улицы
sql29 = """
        UPDATE moscow_buildings
        SET street = '%s'
        WHERE id = %d
        """

#ST_Within(ST_GeomFromEWKT('SRID=4326;POINT(37.617635 55.755814)'),polygon)
#ST_Contains(way, st_transform(st_geomfromtext('POINT(37.511186073182685 55.81656805)', 4326), 3857))

In [2]:
# Functions

def run(cursor, sql):
    cursor.execute(sql)
    for line in cursor.fetchall():
        print(line)
        
def run1(cursor, sql):
    cursor.execute(sql%'Москва')
    connection.commit()
    
def get_streets_from_tags(cursor, sql):
    streets = []
    cursor.execute(sql)
    for line in cursor.fetchall():
        id_ = line[0]
        street = line[1]
        f_pos = street.find("addr:street")
        if f_pos == -1:
            streets.append(["None", id_])
        else:
            street = street[f_pos:]
            street = street[:street.find(",")]
            street = street.split("=>")[1].strip("\"")
            streets.append([street, id_])
    return streets

def load_streets_to_table(streets, connection, cursor, sql):
    for line in streets:
        params = tuple(line)
        cursor.execute(sql % params)
    connection.commit()

In [10]:
# Comands:

#connection, cursor = get_connection("mydatabase", "shurik", "Lampard8", "localhost", "5432")
#print(get_table_names(cursor))
print (get_column_names("streets", cursor))
#cursor.execute(sql)
#connection.commit()

['id', 'name', 'highway', 'z_order', 'way']


In [4]:
# run1(cursor, sql27)
# streets = get_streets_from_tags(cursor, sql28) 
# load_streets_to_table(streets, connection, cursor, sql29)